# 卷積神經網路實作(LeNet-5實作)

# 1

(1) 載入資料：TensorFlow中自MNIST手寫數字識別圖像數據集

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
# 將資料做一個歸一化的動作
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x,[28,28,1])
    y = tf.cast(y, dtype=tf.int32)
    return x, y

batchs = 32

# 載入mnist 資料集 60000張訓練資料 , 10000張測試資料, 每張大小為 28x28
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

(2) 資料打散與封裝

In [ ]:
# 將訓練集資料打散
db = tf.data.Dataset.from_tensor_slices((train_Data, train_Label))
db = db.map(preprocess).shuffle(10000).batch(batchs) # 打散後包裝成批

db_test = tf.data.Dataset.from_tensor_slices((test_Data, test_Label))
db_test = db_test.map(preprocess).batch(batchs)

(3) 模型建置

In [ ]:
# 模型建置
LeNet5Model = Sequential([
    # 第一個卷積層，6個 5x5 卷積核,激勵函數為 relu
    Conv2D(6,kernel_size=5,strides=1,padding='same',activation='relu'),
    # 池化層大小 2x2, 步長 2
    MaxPooling2D(pool_size=2,strides=2),
    # 第二個卷積層，16個 5x5 卷積核, 步長為 1
    Conv2D(16,kernel_size=5,strides=1,padding='same',activation='relu'),
    # 池化層大小 2x2, 步長 2
    MaxPooling2D(pool_size=2,strides=2),
    # 打平層，方便全連接層處理
    Flatten(),
    # 全連接層，120 個節點, 激勵函數為 relu
    Dense(120, activation='relu'),
    # 全連接層，84 個節點, 激勵函數為 relu
    Dense(84, activation='relu'),
    # 全連接層(輸出)，10 個節點, 最後以機率方式呈現
    Dense(10,activation='softmax')
])

In [ ]:
# 指定輸入數據維度
LeNet5Model.build(input_shape=(None, 28, 28, 1))
# 顯示參數量
print(LeNet5Model.summary())

(4) 設定優化器與編譯模型

In [ ]:
# 設定優化器
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# 配置模型  # label 為數字編碼
LeNet5Model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',  # 指定損失函數
                    metrics=['accuracy'])

(5) 訓練模型並追蹤正確率與損失值

In [ ]:
# 訓練模型
hist = LeNet5Model.fit(db,epochs=5, validation_data=db_test)

# AlexNet網路實作

(1) 下載資料集

In [ ]:
from tensorflow.keras.datasets import cifar10
# 載入 cifar10 資料集 50000張訓練資料 , 10000張測試資料, 每張大小為 32x32,3通道
(train_Data, train_Label), (test_Data, test_Label) = cifar10.load_data()
print("train_Data.shape",train_Data.shape)
print("train_Label.shape",train_Label.shape)
print("test_Data.shape",test_Data.shape)
print("test_Label.shape",test_Label.shape)

(2) 數據分割

In [ ]:
# 資料切割, 訓練資料的前面 5000 筆當作是驗證集, 剩下的為測試集
validation_data, validation_label = train_Data[:5000],train_Label[:5000]
train_Data,train_Label= train_Data[5000:],train_Label[5000:]
# 印出訓練資料與驗證資料大小
print("train_Data.shape",train_Data.shape)
print("validation_data.shape",validation_data.shape)

(3) 資料合成與顯示資料及影像

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
CLASS_NAME=["airplane","automobile","bird","cat","deer",
            "dog","frog","horse","ship","truck"]
train_ds = tf.data.Dataset.from_tensor_slices((train_Data,train_Label))
test_ds = tf.data.Dataset.from_tensor_slices((test_Data, test_Label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_data,
                                                    validation_label))

plt.figure(figsize=(5,5))
# 顯示前九張資料影像
for i,(image,label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3,3,1+i)
    plt.imshow(image)
    plt.title(CLASS_NAME[label.numpy()[0]])
    plt.axis('off')
plt.show()

(4) 資料預處理：資料大小轉換與標準化

In [ ]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image,(227,227))
    return image,label

batch_size = 20 #原定360
# 訓練集資料需要打散(shuffle)，打散後分包(batch)
train_ds = train_ds.map(preprocess).shuffle(1000).batch(batch_size=batch_size)
# 驗證集與測試集資料不用打散
validation_ds = validation_ds.map(preprocess).batch(batch_size=batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size=batch_size)

(5) 網路設計

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, \
    Flatten,BatchNormalization,Dropout

model = Sequential([
    # 第一層  卷積層 + BN 層 + 最大池化層
    # filters=48+48=96
    Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),
           activation='relu',input_shape=(227,227,3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3,3),strides=(2,2)),
    # 第二層  卷積層 + BN 層 + 最大池化層
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1),
           activation='relu',padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # 第三層  卷積層 + BN 層
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1),
           activation='relu', padding='same'),
    BatchNormalization(),
    # 第四層  卷積層 + BN 層
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1),
           activation='relu', padding='same'),
    BatchNormalization(),
    # 第五層  卷積層 + BN 層 + 最大池化層
    Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1),
           activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # 展開層    
    Flatten(),     
    Dense(4096,activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

(6) 編譯與訓練網路

In [ ]:
# 編譯與訓練網路
model.compile(optimizer= 'adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
History = model.fit(train_ds,epochs=5,validation_data=validation_ds,   #epochs原定30
                    validation_freq=1)

(7) 評估測試集

In [ ]:
# 評估網路
loss, acc = model.evaluate(test_ds, verbose=0)
print("Test loss :", loss)
print("Test accuracy :", acc)